## Artificial Intelligence Assignment 3
## No3. CNN1 Small and Deep Kernal
## 20132651 Sungjae Lee

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F

In [3]:
## 3 * 3 의 작은 커널을 사용하여 깊은 컨볼루션 신경망을 구현하기 위한 코드입니다.
## conv 층을 새롭게 생성하여 각각을 구분하기 위해 서로 다른 이름을 붙였습니다.
## 진행 과정은 다음과 같습니다.

## conv1 -> conv2 -> relu -> pool -> conv3 -> conv4 -> relu -> pool
## -> view 를 통한 크기변환 -> fc1 -> relu -> fc2 -> relu -> fc3 (10개 클래스 결과)
## FC 라고 쓰여있던 부분은 기존과 동일하게 3개의 fully connected 층을 사용하였습니다.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)        
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.conv4 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

## 컨볼루션 층의 쌓인 모습을 쉽게 볼 수 있도록 각각의 과정을 풀어서 작성하였습니다.
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### (3) 화면 출력 확인 및 학습이 되고 있는지 서술

In [5]:
for epoch in range(2):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss : %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

## 기존의 2번 문제에서의 최종 loss 는 1.226 이었으며, 
## 모든 컨볼루션 층에 대해 32의 출력을 갖는 3 * 3 커널을 새롭게 적용한 결과
## 새로운 CNN 에서 최종 loss 는 1.065 정도로 이전보다 성능이 좋아졌음을 확인할 수 있습니다.

## 마치 VGGNet 이 더 작은 3 * 3 커널을 사용하여 컨볼루션 층을 깊게 쌓은 것처럼 시도한 것이며,
## 그 결과 좁고 깊은 신경망이 더 학습을 잘한다는 점을 알 수 있었습니다.
## 학습의 진행 또한 loss 의 꾸준한 감소 과정을 통해 적절하다는 점을 확인할 수 있습니다.

[1,  1000] loss : 2.301
[1,  2000] loss : 2.211
[1,  3000] loss : 2.023
[1,  4000] loss : 1.888
[1,  5000] loss : 1.772
[1,  6000] loss : 1.661
[1,  7000] loss : 1.588
[1,  8000] loss : 1.509
[1,  9000] loss : 1.530
[1, 10000] loss : 1.463
[1, 11000] loss : 1.451
[1, 12000] loss : 1.394
[2,  1000] loss : 1.334
[2,  2000] loss : 1.315
[2,  3000] loss : 1.271
[2,  4000] loss : 1.297
[2,  5000] loss : 1.223
[2,  6000] loss : 1.196
[2,  7000] loss : 1.179
[2,  8000] loss : 1.142
[2,  9000] loss : 1.183
[2, 10000] loss : 1.152
[2, 11000] loss : 1.122
[2, 12000] loss : 1.065
Finished Training


### (6) 화면 출력 확인 및 일반화 성능 서술

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images :%d %%'%(100 * correct / total))

## 이전의 CNN 에서는 55% 의 Accuracy 가 나타났지만, 이번에는 60% 로 5% 상향된 결과가 나타났습니다.
## 작은 커널을 이용하여 더 깊은 층을 쌓은 결과, 조금 더 나은 일반화 성능을 얻었다 볼 수 있습니다.
## 하지만 60%는 여전히 높은 성능이 아니기 때문에 성능을 높이기 위해서 더 많은 학습 데이터로 학습을 진행하거나
## 배치 정규화, Adam Optimizer 등의 발전된 최적화 방안을 생각해 보아야 합니다.

Accuracy of the network on the 10000 test images :60 %


In [7]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

### (7) 화면 출력 확인 및 부류별 분류기의 성능 서술

In [8]:
## 앞에서와 마찬가지로 부류별 성능을 출력합니다.
## 이번에 높은 성능의 부류는 Horse, Ship, Car 로 순서는 바뀌었지만 동일한 개체들입니다.
## 이번에 낮은 성능의 부류는 Bird, Dog, Frog, Cat 입니다.
## 이로 미루어 보아, 특정 부류에 대해서만 일정하게 분류기의 성능이 높고 낮음을 알 수 있습니다.

for i in range(10):
    print('Accuracy of %5s: %2d %%'%(classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane: 59 %
Accuracy of   car: 80 %
Accuracy of  bird: 33 %
Accuracy of   cat: 57 %
Accuracy of  deer: 64 %
Accuracy of   dog: 26 %
Accuracy of  frog: 55 %
Accuracy of horse: 80 %
Accuracy of  ship: 81 %
Accuracy of truck: 64 %
